In [1]:
import generator

ImportError: attempted relative import with no known parent package

In [43]:
w = 416
s = 2
p = 1
d = 1
k = 3
o = 1
(w-1)*s-2*p+d*(k-1)+o+1

832

In [47]:
"{}".format(["aaa",2,3])

"['aaa', 2, 3]"

In [ ]:
16,  208,  208],
       [   1,   24,  104,  104],
       [   1,   32,   52,   52],
       [   1,   64,   26,   26],
       [   1,   96,   26,   26],
       [   1, 1280

In [131]:
import torch, torchvision
import os

import torch
import torch.nn
from aimaker.models.base_model import BaseModel
from torch.nn import functional as F
import torch.nn as nn
from MobileNetV2 import MobileNetV2
from numpy import *

class Output(nn.Module):
    def __init__(self, in_channels, out_channels=3, kernel_size=3, stride=1, padding=1, act=nn.Tanh()):
        super(Output, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            act
        )

    def forward(self, X): 
        return self.conv(X)


class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels=3, kernel_size=3, stride=2, padding=1, output_padding=1, act=nn.PReLU()):
        super(UpSample, self).__init__()
        self.conv = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=output_padding),
            nn.InstanceNorm2d(out_channels),
            act
        )

    def forward(self, X): 
        return self.conv(X)

class PyramidModelGenerator():
    def __init__(self):
        #super(PyramidModelGenerator, self).__init__(settings)
        self.model_path = os.path.join('/'.join(os.getcwd().split('/')[:-1]), '../trained_models/mobilenet_v2.pth.tar')
        state_dict = torch.load(self.model_path, map_location=torch.device('cpu'))
        net = MobileNetV2()
        net.load_state_dict(state_dict)
        self.feature = torch.nn.Sequential(*list(list(net.children())[0:1][0].children()))
        #self.feat_lst = array(self.feature)[array([1,3,6,10,13,18])].tolist()
        self.e_lst = [1,3,6,10,13,18]
        #self.up_lst  = [UpSample(x,y, kernel_size=k, stride=m, padding=p).to(self.gpu_ids[0]) for x,y,k,m,p in zip(self.out_channels_lst[:-1],self.in_channels_lst[1:], self.k_lst, self.m_lst, self.pad_lst)]
        self.output = Output(18)
        self._genUpsample()

    def _genUpsample(self):
        self.up_lst = []
        self.up_1 = self._getUp(1, 16)
        self.up_lst += [self.up_1] 
        self.up_2 = self._getUp(2, 24)
        self.up_lst += [self.up_2]
        self.up_3 = self._getUp(3, 32)
        self.up_lst += [self.up_3]
        self.up_4 = self._getUp(4, 64)
        self.up_lst += [self.up_4]
        self.up_5 = self._getUp(4, 96)
        self.up_lst += [self.up_5]
        self.up_6 = self._getUp(5, 1280)
        self.up_lst += [self.up_6]

    def _getUp(self, n_iter, in_channels):
        lst = []
        for i in range(n_iter-1):
            lst += [UpSample(in_channels, in_channels)]
        lst += [UpSample(in_channels)]
        return torch.nn.Sequential(*lst)
    
    def _featureAggregation(self, X):
        lst = []
        n = 0
        for i in range(len(self.feature)):
            X = self.feature[i](X)
            if i in self.e_lst:
                lst += [self.up_lst[n](X)]
                n += 1
        return self.output(torch.cat(lst, dim=1))
    
    def _bottoleNeck(self, X):
        return X
    
    def _aggregate(self, X_segment, X_bottleneck):
        return torch.cat([X_segment, X_bottleneck], dim=1)
    
    def _segment(self, X_segment):
        X = self.segment_bottleNeck(X_segment)
        return X
    
    def _outputPoints(self, X):
        x = x.max(dim=2)[0].argmax(2)
        y = X.max(dim=3)[0].argmax(2)
        return torch.cat([x[:,:, None],y[:,:,None]],dim=2)

    def forward(self, X):
        X = self._featureAggregation(X)
        X_bottleneck = self._bottleNeck(X)
        X_segment, map_segment = self._segment(X)
        X = self._aggregate(X_segment, X_bottleneck)
        map_point = self._outputPoints(X)
        X_point = self._convertCoordinate(map_point)
        trifocal_tensor = self._genTrifocalTensor(X_point)
        return map_point, X_point, map_segment
        
        

In [158]:
a,b = torch.Tensor(random.randn(1,52,128,128)),torch.Tensor(random.randn(2,10).T)

In [159]:
a.shape

torch.Size([1, 52, 128, 128])

In [168]:
.shape

torch.Size([1, 52, 2])

In [163]:
x = a.max(dim=2)[0].argmax(2)
y = a.max(dim=3)[0].argmax(2)

In [136]:
net = PyramidModelGenerator()

In [139]:
ret = net._featureAggregation(torch.Tensor(1,3,416,416))

In [138]:
ret.shape

torch.Size([1, 3, 416, 416])

In [54]:
for param in net.parameters():
    print(param.requires_grad)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [13]:
ls

MobileNetV2.py*  Untitled.ipynb  __pycache__/  generator.py


In [14]:
state_dict = torch.load("../../trained_models/mobilenet_v2.pth.tar")

In [23]:
state_dict['features.0.1.weight'].shape

torch.Size([32])

In [31]:
net.load_state_dict(state_dict)
feature = torch.nn.Sequential(*list(list(net.children())[0:1][0].children()))

In [35]:
lst = []
X = torch.Tensor(1,3,416,416)
for n in feature:
    X = n(X)
    lst += [X.shape]
    

In [37]:
from numpy import *

In [94]:
array(lst)[array([1,3,6,10,13,18])]

array([[   1,   16,  208,  208],
       [   1,   24,  104,  104],
       [   1,   32,   52,   52],
       [   1,   64,   26,   26],
       [   1,   96,   26,   26],
       [   1, 1280,   13,   13]])

[InvertedResidual(
   (conv): Sequential(
     (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
     (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): ReLU6(inplace)
     (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   )
 ), InvertedResidual(
   (conv): Sequential(
     (0): Conv2d(24, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): ReLU6(inplace)
     (3): Conv2d(144, 144, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=144, bias=False)
     (4): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (5): ReLU6(inplace)
     (6): Conv2d(144, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (7): BatchNorm2d(24, eps=1e-05, momentum=0.1, a

In [43]:
w = 416
s = 2
p = 1
d = 1
k = 3
o = 1
(w-1)*s-2*p+d*(k-1)+o+1

832

In [45]:
(torch.Tensor(1,1,416,416)).shape

torch.Size([1, 1, 832, 832])

In [22]:
import pprint 
pprint.pprint(state_dict.keys())

odict_keys(['features.0.0.weight', 'features.0.1.weight', 'features.0.1.bias', 'features.0.1.running_mean', 'features.0.1.running_var', 'features.1.conv.0.weight', 'features.1.conv.1.weight', 'features.1.conv.1.bias', 'features.1.conv.1.running_mean', 'features.1.conv.1.running_var', 'features.1.conv.3.weight', 'features.1.conv.4.weight', 'features.1.conv.4.bias', 'features.1.conv.4.running_mean', 'features.1.conv.4.running_var', 'features.2.conv.0.weight', 'features.2.conv.1.weight', 'features.2.conv.1.bias', 'features.2.conv.1.running_mean', 'features.2.conv.1.running_var', 'features.2.conv.3.weight', 'features.2.conv.4.weight', 'features.2.conv.4.bias', 'features.2.conv.4.running_mean', 'features.2.conv.4.running_var', 'features.2.conv.6.weight', 'features.2.conv.7.weight', 'features.2.conv.7.bias', 'features.2.conv.7.running_mean', 'features.2.conv.7.running_var', 'features.3.conv.0.weight', 'features.3.conv.1.weight', 'features.3.conv.1.bias', 'features.3.conv.1.running_mean', 'fe